## [AutoML]訓練済みモデルをローカルで実行する

> **Point** <br>✓ Google Cloud AutoMLで訓練済みのモデルをSavedmodel形式(拡張子 .pb)でダウンロードできる.<br>✓ ローカルにTensorflow実行環境がある場合, 訓練済みモデルによる予測をローカルで実行できる.


---

### TensorFlowのインストール

✅ [tensorflow](https://www.tensorflow.org/install?hl=ja)



In [34]:
!pip install --upgrade pip
!pip install tensorflow
!pip install matplotlib
!pip install opencv-python
!pip install  pip install ipywidgets

     ---------------------------------------- 0.0/38.2 MB ? eta -:--:--
     ---------------------------------------- 0.4/38.2 MB 8.9 MB/s eta 0:00:05
     - -------------------------------------- 1.1/38.2 MB 11.5 MB/s eta 0:00:04
     -- ------------------------------------- 2.0/38.2 MB 14.3 MB/s eta 0:00:03
     --- ------------------------------------ 3.1/38.2 MB 16.7 MB/s eta 0:00:03
     ---- ----------------------------------- 4.6/38.2 MB 19.4 MB/s eta 0:00:02
     ------ --------------------------------- 6.2/38.2 MB 21.9 MB/s eta 0:00:02
     -------- ------------------------------- 7.7/38.2 MB 23.3 MB/s eta 0:00:02
     --------- ------------------------------ 9.2/38.2 MB 25.5 MB/s eta 0:00:02
     ---------- ---------------------------- 10.5/38.2 MB 27.3 MB/s eta 0:00:02
     ----------- --------------------------- 11.6/38.2 MB 29.7 MB/s eta 0:00:01
     ------------- ------------------------- 12.8/38.2 MB 29.8 MB/s eta 0:00:01
     -------------- ------------------------ 14.1

In [1]:
import tensorflow as tf
print(tf.__version__)

2.12.0


---

### SavedModelファイルを用いて予測を実行する

**参考**

✓ [SavedModel 形式の使用](https://www.tensorflow.org/guide/saved_model?hl=ja)  
✓ [stack overflow](https://stackoverflow.com/questions/61379555/use-saved-model-pb-from-google-cloud-automl-to-predict-in-python-locally)

In [5]:
# モデルの読み込み
# saved_model.pbを格納したフォルダを指定
SAVED_MODEL_DIR = './model'

loaded = tf.saved_model.load(SAVED_MODEL_DIR)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [4]:
# 読み込んだモデルで画像分類を行う
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
from ipywidgets import interact

@interact(file=['{}.jpg'.format(str(i))  for i in range(1, 10, 1)])
def predict(file):
    # 画像読み込み
    img = cv2.imread('./images/{}'.format(file))
    plt.imshow(img)
    plt.axis('off')

    rst,bts = cv2.imencode('.jpg', img)
    bimg = [bts.tobytes()]

    # 各クラスへの分類確率を算出
    infer = loaded.signatures["serving_default"]
    result = infer(key=tf.constant('123'), image_bytes=tf.constant(bimg))

    print("{} \t\t{}".format("ラベル", "確率 (単位:%)"))
    for label, score in zip(result['labels'].numpy()[0], result['scores'].numpy()[0]):
        print("{} \t{:.2f}%".format(label, score * 100))

interactive(children=(Dropdown(description='file', options=('1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jp…